In [1]:
import numpy as np
import pandas as pd
from numpy.random import seed
from numpy.random import randint
import string

seed(1)

In [2]:
total_clients = 1500
min_mv = 100000.0
max_mv = 10000000.0
min_received_pct = 1.0
max_received_pct = 65.0
min_diversity = 1.0
max_diversity = 60.0

In [3]:
def random(min, max):
    array = randint(min, max, 1)
    return float(array[0])

def get_eci():
    strs = string.ascii_letters
    index = random(0.1, 25.0)
    l = strs[int(index)]
    n = randint(1, 1000000, 1010000)
    return '{0}00{1}'.format(l, n[0])

def get_portfolio_diversity(max_mv, mv):
    pct = max_mv/mv
    div_amt = random(min_diversity, max_diversity)/100
    diversity_val = pct * div_amt
    return diversity_val

def get_received(i, totals):
    total = totals[int(i)]
    fractn = random(min_received_pct, max_received_pct)/float(100)
    return fractn * total

def received_margin_call(total, received_mv, diversity):
    less_totals = (received_mv/total) < 0.1
    less_diversity = (diversity/max_diversity) < 0.1
    called = 1 if less_totals and less_diversity else 0
    return called
        

In [4]:
client_indices = np.arange(total_clients)

console = lambda label, array: print('\n..................{0}...........\n length = {1}\n{2}'.format(label, len(array), array))

# ECIs
f = lambda eci: get_eci()
ecis = np.array([f(eci) for eci in client_indices])

#TOTALs
f = lambda i: random(min_mv, max_mv)
totals = np.array([f(i) for i in client_indices])

# RECEIVED
f = lambda i: get_received(i, totals)
received = np.array([f(i) for i in np.nditer(client_indices)])

# PLEDGED
f = lambda i, t: t - received[int(i)]
pledged = np.array([f(i, totals[i]) for i in np.nditer(client_indices)])

# UNPLEDGED
f = lambda i: totals[int(i)] - received[int(i)]
unpledged = np.array([f(i) for i in np.nditer(client_indices)])

# PORTFOLIO_DIVERSITY
f = lambda i: get_portfolio_diversity(max_mv, totals[int(i)])
diversity = np.array([f(i) for i in np.nditer(client_indices)])

# MARGIN_CALLED
f = lambda i: received_margin_call(totals[i], received[i], diversity[i])
margin_called = np.array([f(i) for i in np.nditer(client_indices)])

# FINAL DATA

client_data = {
    'ecis': ecis.flatten(),
    'totals': totals.flatten(),
    'received': received.flatten(),
    'pledged': pledged.flatten(),
    'unpledged': unpledged.flatten(),
    'diversity': diversity.flatten(),
    'margin_called': margin_called.flatten()
}
client_data = pd.DataFrame(client_data)
console('FINAL DATA', client_data)
print('{0}'.format(client_data[client_data.margin_called == 1].shape[0]))
client_data.to_csv(path_or_buf='client_data2.csv', index=False)


..................FINAL DATA...........
 length = 1500
           ecis     totals    received     pledged   unpledged  diversity  \
0     f00491756  2153297.0   129197.82  2024099.18  2024099.18   1.300332   
1     h00943220  3678610.0  1545016.20  2133593.80  2133593.80   0.407763   
2     d00767193  2343829.0   515642.38  1828186.62  1828186.62   2.517248   
3     n00356473  4477480.0   850721.20  3626758.80  3626758.80   0.111670   
4     v00534983  5308213.0   743149.82  4565063.18  4565063.18   0.810065   
5      f0084842  6143695.0  2641788.85  3501906.15  3501906.15   0.520859   
6     v00325091  2664893.0   986010.41  1678882.59  1678882.59   1.088224   
7     o00335428  2200889.0  1100444.50  1100444.50  1100444.50   0.863288   
8     o00597602  1695627.0   440863.02  1254763.98  1254763.98   0.943604   
9     w00321767  6647068.0  4121182.16  2525885.84  2525885.84   0.105310   
10    j00395747  4919725.0  1820298.25  3099426.75  3099426.75   0.386200   
11    s00591123  487